In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F,CrossEntropyLoss
from torch import optim
from torch.nn import LSTM,Linear
import math
import copy
from typing import Callable, Any, Optional, Tuple, List

In [109]:
def clones(module, N):
    """克隆N层一模一样的
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class Api2resultmodel(nn.Module):
    '''Api--->result Sequence model'''
    def __init__(self,hidden_size,result_channels,*args, **kwargs) -> None:
        super(Api2resultmodel,self).__init__()
        self.Linear=nn.Linear(hidden_size,result_channels)
    def forward(self,x):
        results_proba=F.softmax(self.Linear(x))
        return results_proba
   

class Api_Sequence2Result_Sequence_finddif_model(nn.Module):
    '''Api_Sequence to result_Sequence_to next_Api_model'''
    def __init__(self,Api_dim,Api_class_nums,windows_size,hidden_layers1,hidden_layers2,result_channels,lstm1_units,lstm2_units,lstm3_units,lstm4_units,Api2result_Basic: Optional[Callable[..., nn.Module]] = None,**kwargs:Any) -> None:
        super(Api_Sequence2Result_Sequence_finddif_model,self).__init__()
        if Api2result_Basic==None:
            Api2result_Basic=Api2resultmodel(hidden_layers1,result_channels)
        
        #####first_dim=batch_size
        self.lstm1=nn.LSTM(input_size=Api_dim,hidden_size=hidden_layers1,num_layers=lstm1_units,batch_first=True,**kwargs)        
       
        self.Linear_Block=clones(Api2result_Basic,windows_size)

        self.lstm2=nn.LSTM(input_size=result_channels,hidden_size=hidden_layers2,num_layers=lstm2_units,batch_first=True,**kwargs)

        self.apilstm1=nn.LSTM(input_size=Api_dim,hidden_size=hidden_layers2,num_layers=lstm3_units,batch_first=True,**kwargs)

        self.apilstm2=nn.LSTM(input_size=Api_dim,hidden_size=hidden_layers2,num_layers=lstm4_units,batch_first=True,**kwargs)

        self.Flatten1=nn.Flatten()

        self.Dropout=nn.Dropout(0.1)

        self.ApiLinear=nn.Linear(hidden_layers2*windows_size+(windows_size+(windows_size)//2)*hidden_layers2,Api_class_nums)
    

    def forward(self,window_sizedata,Dropout=True):
        #forward 
        #faster compute
        window_sizedata=torch.tensor(window_sizedata) if type(window_sizedata)!=torch.tensor else window_sizedata
        if window_sizedata.dim()==2:
            window_sizedata=window_sizedata.reshape(1,window_sizedata.size(0),window_sizedata.size(1))
        # batch forward 
         #to get output of everystep and the outputdim of everystep is hidden_size     
        window_sequence=self.lstm1(window_sizedata)[0]    
            
        api2result_sequence=torch.tensor([l(window_sequence[:,idx,:]).tolist() for idx,l in enumerate(self.Linear_Block)]).permute(1,0,2)
        #utilize info of api->result_sequence to next lstm
        result_sequence=self.lstm2(api2result_sequence)[0]
         
        #bigger windowsize lstm
        coarse_Apisequence_output=self.apilstm1(window_sizedata)[0]

        #smaller windowsize lstm
        fine_Apisequence_output=self.apilstm2(window_sizedata[:,-(window_sizedata).size(1)//2:,])[0]   
        
        Apisequence=torch.concat([coarse_Apisequence_output,fine_Apisequence_output],axis=1) 
        
        #utilize result_sequence to class
        result_sequence_flatten=self.Flatten1(result_sequence)

        api_sequence_flatten=self.Flatten1(Apisequence)
        
        all_suquence=torch.concat([result_sequence_flatten,api_sequence_flatten],axis=1) 
        
        if Dropout:
        #Dropout 
            allsequence_afdpt=self.Dropout(all_suquence)
        #class
            Api_class_proba=F.softmax(self.ApiLinear(allsequence_afdpt))
        else:
            Api_class_proba=F.softmax(self.ApiLinear(all_suquence))
        return  api2result_sequence,Api_class_proba


In [130]:


class Lstmlocal_find_sequence_dif_model(nn.Module):
    def __init__(self, Api_dim,Api_class,hidden_layers,bigger_window_size=10,smaller_window_size=5, **kwargs) -> None:
        super(Lstmlocal_find_sequence_dif_model,self).__init__()
        #define windows size 
        self.bigger_window_size=bigger_window_size
        
        self.smaller_window_size=smaller_window_size

        #window_size is 10
        self.apilstm1=nn.LSTM(input_size=Api_dim,hidden_size=hidden_layers,batch_first=True,**kwargs)
        #window_size is 5
        self.apilstm2=nn.LSTM(input_size=Api_dim,hidden_size=hidden_layers,batch_first=True,**kwargs)
        
        self.Dropout=nn.Dropout(0.1)
        
        self.Flatten=nn.Flatten()

        self.classlinear=nn.Linear((bigger_window_size+smaller_window_size)*hidden_layers,Api_class)

    def forward(self,window_sizedata,Dropout=True):
        #faster compute
        window_sizedata=torch.tensor(window_sizedata) if type(window_sizedata)!=torch.tensor else window_sizedata
        if window_sizedata.dim()==2:
            window_sizedata=window_sizedata.reshape(1,window_sizedata.size(0),window_sizedata.size(1))
        
        
        bigger_window_size=self.bigger_window_size

        smaller_window_size=self.smaller_window_size

        biggerlstmoutput=self.apilstm1(window_sizedata[:,-bigger_window_size:,])[0]

        smallerlstmouput=self.apilstm2(window_sizedata[:,-smaller_window_size:,])[0]

        if Dropout:

            biggerlstmoutput=self.Dropout(biggerlstmoutput)

            smallerlstmouput=self.Dropout(smallerlstmouput)
        
        api_sequence=torch.concat([biggerlstmoutput,smallerlstmouput],axis=1)


        flatten_api_sequence=self.Flatten(api_sequence)

        output=F.softmax(self.classlinear(flatten_api_sequence))
        return  output


In [133]:
Lstmlocal_find_sequence_dif_model(Api_dim=8,Api_class=15948,hidden_layers=20,bigger_window_size=10,smaller_window_size=5)(torch.randn(100,8)).shape

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3364\2526116284.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  window_sizedata=torch.tensor(window_sizedata) if type(window_sizedata)!=torch.tensor else window_sizedata
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3364\2526116284.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output=F.softmax(self.classlinear(flatten_api_sequence))


torch.Size([1, 15948])

In [128]:
class Conv1dlocal_find_sequence_dif_model(nn.Module):
    def __init__(self, windowsize,Api_dim,Api_classnum,outchannels1,outchannels2,*args, **kwargs) -> None:
        super().__init__()
        #bigger_conv1d to find local_dif
        self.bigger_conv1d=nn.Conv1d(in_channels=Api_dim,out_channels=outchannels1,kernel_size=5,padding=2,bias=True)
        
        self.bn1=nn.BatchNorm1d(num_features=outchannels1,eps=0.001)

        #smaller_cov1d to find local_dif
        self.smaller_conv1d=nn.Conv1d(in_channels=Api_dim,out_channels=outchannels2,kernel_size=3,padding=1,bias=True)

        self.bn2=nn.BatchNorm1d(num_features=outchannels2,eps=0.001)

        self.flatten=nn.Flatten()

        self.classlinear=nn.Linear(windowsize*(outchannels1+outchannels2),Api_classnum)

    def forward(self,window_sizedata):
        #faster compute 
        window_sizedata=torch.tensor(window_sizedata) if type(window_sizedata)!=torch.tensor else window_sizedata
        if window_sizedata.dim()==2:
            window_sizedata=window_sizedata.reshape(1,window_sizedata.size(0),window_sizedata.size(1))
        
        #need permuter window_sizedata to (batch_size,channels,api_windowsize)
        window_sizedata=window_sizedata.permute(0,2,1)
            
        bigger_conv=self.bn1(self.bigger_conv1d(window_sizedata))

        smaller_conv=self.bn2(self.smaller_conv1d(window_sizedata))

        all_conv=torch.concat([bigger_conv,smaller_conv],axis=1).permute(0,2,1)
        

        all_conv=self.flatten(all_conv)

        return F.softmax(self.classlinear(all_conv))